In [1]:
import cv2
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
import numpy as np

In [8]:
origin_features_folder = "/home/user/Desktop/lanyard_segmentation/guardrail/orig_features"
origin_labels_folder = "/home/user/Desktop/lanyard_segmentation/guardrail/orig_labels"
orig_added_features_folder="/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features"
orig_added_labels_folder="/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/labels"

# training data
# features
train_features_folder = "/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features"
train_features_10crop = "/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped"
train_features_10crop_resized = "/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized"

train_added_features_folder = "/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features"
train_added_features_10crop = "/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features_cropped"
train_added_features_10crop_resized = "/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features_cropped_resized"

# labels
train_labels_folder = "/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels"
train_labels_10crop = "/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped"
train_labels_10crop_resized = "/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized"

train_added_labels_folder = "/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/labels"
train_added_labels_10crop = "/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/labels_cropped"
train_added_labels_10crop_resized = "/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/labels_cropped_resize"

In [4]:
def read_images(root_path):
  for root, dirs, files in os.walk(root_path):
    print("path：", root)
    print("directory：", dirs)
    print("file：", files)
  imgs = [str(root + "/" + file_name) for file_name in files]

  imgs.sort()
  print(len(imgs))
  return imgs

In [5]:
def resize_imgs(path_list, save_path, resize_size=(320, 320), is_feature=True):
  c = 0
  for path in path_list:
    c += 1
    img = cv2.imread(path)
    new_path = save_path + '/' + os.path.basename(path)
    if is_feature:
      img = cv2.resize(img, resize_size, interpolation=cv2.INTER_AREA)
    else:
      img = cv2.resize(img, resize_size, interpolation=cv2.INTER_NEAREST)
    cv2.imwrite(new_path, img)
    print(new_path+" saved!")

  print("Saved {} images".format(str(c)))

In [6]:
def save_img(path_list, save_folder):
  c = 0
  for path in path_list:
    c+=1
    img = cv2.imread(path)
    save_path = save_folder + '/' + path[-8:]
    cv2.imwrite(save_path, img)
    print(save_path+" saved!")
  
  print("total saved: {} images".format(str(c)))

In [7]:
def ten_crops(f_img_path, f_save_path, l_img_path, l_save_path, crop_size=0.7):

  if crop_size >= 1:
    raise Exception('crop_size should be smaller than 1')
  
  count = 0
  f_img = cv2.imread(f_img_path)
  l_img = cv2.imread(l_img_path)
  basename = os.path.basename(f_img_path)
  print('Proccessing ' + basename + "...")
  file_name = basename.split(".")
  size = f_img.shape


  x = size[0]
  y = size[1]
  w = round(x * crop_size)
  h = round(y * crop_size)
  x_mid = round(((1 - crop_size) / 2) * x)
  y_mid = round(((1 - crop_size) / 2) * y)
  start = [[0,w,0,h], [0,w,y-h,y], [x-w,x,0,h], [x-w,x,y-h,y], [x_mid,x_mid+w,y_mid,y_mid+h]]


  for i in range(5):
    # feature crop
    crop_f_img = f_img[start[i][0]:start[i][1], start[i][2]:start[i][3]]
    crop_f_full_path = f_save_path + '/' + file_name[0] + '_' + str(i*2+1) + '.' + file_name[1]


    # flip
    crop_f_img_flip = cv2.flip(crop_f_img, 1)
    crop_f_flip_full_path = f_save_path + '/' + file_name[0] + '_' + str(i*2+2) + '.' + file_name[1]


    # labels crop
    crop_l_img = l_img[start[i][0]:start[i][1], start[i][2]:start[i][3]]
    crop_l_full_path = l_save_path + '/' + file_name[0] + '_' + str(i*2+1) + '.' + file_name[1]


    # flip
    crop_l_img_flip = cv2.flip(crop_l_img, 1)
    crop_l_flip_full_path = l_save_path + '/' + file_name[0] + '_' + str(i*2+2) + '.' + file_name[1]


    # save
    if len(np.unique(crop_l_img)) > 1:
      cv2.imwrite(crop_f_full_path, crop_f_img)
      cv2.imwrite(crop_l_full_path, crop_l_img)
      count += 1

    if len(np.unique(crop_l_img_flip)) > 1:
      cv2.imwrite(crop_f_flip_full_path, crop_f_img_flip)
      cv2.imwrite(crop_l_flip_full_path, crop_l_img_flip)
      count += 1
  
  print('total images saved: {}'.format(str(count)))


In [9]:
f_imgs = read_images(orig_added_features_folder)
l_imgs = read_images(orig_added_labels_folder)

path： /home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features
directory： []
file： ['0075.png', '0086.png', '0074.png', '0079.png', '0083.png', '0072.png', '0088.png', '0077.png', '0084.png', '0078.png', '0076.png', '0081.png', '0082.png', '0073.png', '0080.png', '0087.png', '0085.png']
17
path： /home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/labels
directory： []
file： ['0075.png', '0086.png', '0074.png', '0079.png', '0083.png', '0072.png', '0088.png', '0077.png', '0084.png', '0078.png', '0076.png', '0081.png', '0082.png', '0073.png', '0080.png', '0087.png', '0085.png']
17


In [10]:
for i in range(len(f_imgs)):
  ten_crops(f_imgs[i], train_added_features_10crop, l_imgs[i], train_added_labels_10crop, 0.7)

Proccessing 0072.png...
total images saved: 10
Proccessing 0073.png...
total images saved: 10
Proccessing 0074.png...
total images saved: 10
Proccessing 0075.png...
total images saved: 10
Proccessing 0076.png...
total images saved: 10
Proccessing 0077.png...
total images saved: 10
Proccessing 0078.png...
total images saved: 10
Proccessing 0079.png...
total images saved: 10
Proccessing 0080.png...
total images saved: 10
Proccessing 0081.png...
total images saved: 10
Proccessing 0082.png...
total images saved: 10
Proccessing 0083.png...
total images saved: 10
Proccessing 0084.png...
total images saved: 10
Proccessing 0085.png...
total images saved: 10
Proccessing 0086.png...
total images saved: 10
Proccessing 0087.png...
total images saved: 10
Proccessing 0088.png...
total images saved: 10


In [11]:
train_crop_features_added = read_images(train_added_features_10crop)
train_crop_labels_added = read_images(train_added_labels_10crop)

path： /home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features_cropped
directory： []
file： ['0079_7.png', '0087_7.png', '0072_5.png', '0075_2.png', '0075_8.png', '0078_1.png', '0074_6.png', '0079_6.png', '0080_5.png', '0078_4.png', '0082_10.png', '0079_9.png', '0083_9.png', '0086_10.png', '0079_4.png', '0078_6.png', '0083_7.png', '0081_8.png', '0087_2.png', '0086_2.png', '0081_10.png', '0072_2.png', '0077_1.png', '0087_6.png', '0076_6.png', '0072_1.png', '0077_2.png', '0080_3.png', '0088_6.png', '0081_5.png', '0082_3.png', '0072_10.png', '0072_7.png', '0078_2.png', '0074_9.png', '0086_5.png', '0076_10.png', '0088_9.png', '0080_2.png', '0073_6.png', '0086_9.png', '0076_3.png', '0073_9.png', '0082_6.png', '0080_4.png', '0084_6.png', '0075_4.png', '0078_10.png', '0086_6.png', '0073_4.png', '0085_3.png', '0084_9.png', '0085_9.png', '0083_10.png', '0079_2.png', '0083_2.png', '0074_2.png', '0073_2.png', '0074_10.png', '0083_5.png', '0083_3.png', '0077_6.png', '0082_8.png', '0086

In [12]:
resize_imgs(train_crop_features_added, train_added_features_10crop_resized, resize_size=(320, 320))
resize_imgs(train_crop_labels_added, train_added_labels_10crop_resized, resize_size=(320, 320), is_feature=False)

/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features_cropped_resized/0072_1.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features_cropped_resized/0072_10.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features_cropped_resized/0072_2.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features_cropped_resized/0072_3.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features_cropped_resized/0072_4.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features_cropped_resized/0072_5.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features_cropped_resized/0072_6.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features_cropped_resized/0072_7.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features_cropped_resized/0072_8.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/fe

/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features_cropped_resized/0080_6.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features_cropped_resized/0080_7.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features_cropped_resized/0080_8.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features_cropped_resized/0080_9.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features_cropped_resized/0081_1.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features_cropped_resized/0081_10.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features_cropped_resized/0081_2.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features_cropped_resized/0081_3.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features_cropped_resized/0081_4.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/fe

/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features_cropped_resized/0088_5.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features_cropped_resized/0088_6.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features_cropped_resized/0088_7.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features_cropped_resized/0088_8.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/features_cropped_resized/0088_9.png saved!
Saved 170 images
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/labels_cropped_resize/0072_1.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/labels_cropped_resize/0072_10.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/labels_cropped_resize/0072_2.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/labels_cropped_resize/0072_3.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_

/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/labels_cropped_resize/0081_9.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/labels_cropped_resize/0082_1.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/labels_cropped_resize/0082_10.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/labels_cropped_resize/0082_2.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/labels_cropped_resize/0082_3.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/labels_cropped_resize/0082_4.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/labels_cropped_resize/0082_5.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/labels_cropped_resize/0082_6.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/labels_cropped_resize/0082_7.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/dataset_72/labels_cropped_resize/0082_8.

In [8]:
train_crop_features = read_images(train_features_10crop)
train_crop_labels = read_images(train_labels_10crop)

path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0048_9.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0025_8.png', '0020_2.png', '0064_4.png', '0047_1.png', '0066_4.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0030_8.png', '0023_2.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0050_8.png', '0067_7.png', '0006_5.png', '0040_9.png', '0005_9.png', '0013_3.png', '0025_3.png', '0054_2.png', '0024_8.png', '0029_6.png', '0023_9.

In [9]:
resize_imgs(train_crop_features, train_features_10crop_resized, resize_size=(320, 320))
resize_imgs(train_crop_labels, train_labels_10crop_resized, resize_size=(320, 320), is_feature=False)

/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0001_1.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0001_10.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0001_2.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0001_3.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0001_4.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0001_5.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0001_6.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0001_7.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0001_8.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/fe

/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0009_3.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0009_4.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0009_5.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0009_6.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0009_7.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0009_8.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0009_9.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0010_1.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0010_10.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/fe

/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0018_1.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0018_10.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0018_2.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0018_3.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0018_4.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0018_5.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0018_6.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0018_7.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0018_8.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/fe

/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0026_4.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0026_5.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0026_6.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0026_7.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0026_8.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0026_9.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0027_1.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0027_10.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0027_2.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/fe

/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0034_4.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0034_5.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0034_6.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0034_7.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0034_8.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0034_9.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0035_1.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0035_10.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped_resized/0035_2.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/fe

...

/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0032_1.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0032_10.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0032_2.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0032_3.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0032_4.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0032_5.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0032_6.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0032_7.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0032_8.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resiz

/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0040_4.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0040_5.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0040_6.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0040_7.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0040_8.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0040_9.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0041_1.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0041_10.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0041_2.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resiz

/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0048_9.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0049_1.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0049_10.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0049_2.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0049_3.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0049_4.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0049_5.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0049_6.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0049_7.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resiz

/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0057_3.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0057_4.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0057_5.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0057_6.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0057_7.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0057_8.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0057_9.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0058_1.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0058_10.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resiz

/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0065_7.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0065_8.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0065_9.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0066_1.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0066_10.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0066_2.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0066_3.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0066_4.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resized/0066_5.png saved!
/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped_resiz